In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
def rfeFeature(indep_X,dep_Y,n):
    rfeList=[]
    
    RF = RandomForestRegressor(random_state=0)
    DT = DecisionTreeRegressor(random_state=0)
    svr_model= SVR(kernel="linear",C=3000)
    rfemodellist=[svr_model,RF,DT]
    for i in rfemodellist:
        print(i)
        log_rfe = RFE(i,n)
        log_fit = log_rfe.fit(indep_X,dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfeList.append(log_rfe_feature)
        return rfeList

In [3]:
def split_scaler(indep_X,dep_Y):
    X_train,X_test,y_train,y_test=train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc=StandardScaler()
    X_train=sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    return X_train,X_test,y_train,y_test

In [4]:
def rscore_prediction(regressor,X_test,y_test):
    y_pred=regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r_score=r2_score(y_test,y_pred)
    return regressor,r_score
def svm_linear(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel="linear",C=3000)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score
def Decision(X_train,y_train,X_test,y_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score
def random(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(random_state=0)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score

In [5]:
def rfe_regression(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf):
    data = { 'SVMl': accsvml, 'Decision': accdes, 'Random': accrf}
    rfedataframe=pd.DataFrame(data, index=['SVR','Random','Decision Tree'])
    return rfedataframe

In [6]:
dataset1=pd.read_csv("prepkidney_data.csv",index_col=None)
df2=pd.get_dummies(dataset1,drop_first=True)

In [7]:
indep_X=df2.drop("classification_ckd\t",1)
dep_Y= df2['classification_ckd\t']

In [ ]:
rfeList=rfeFeature(indep_X,dep_Y,2)


accsvml=[]
accdes=[]
accrf=[]

SVR(C=3000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
for i in rfeList:
    X_train,X_test,y_train,y_test= split_scaler(i,dep_Y)
    
    regressor,r_score= svm_linear(X_train,y_train,X_test,y_test)
    accsvml.append(r_score)
    regressor,r_score= Decision(X_train,y_train,X_test,y_test)
    accdes.append(r_score)
    regressor,r_score= random(X_train,y_train,X_test,y_test)
    accrf.append(r_score)

    result= rfe_regression(accsvml,accdes,accrf)